In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import numpy as np
from rtseg.utils.param_io import load_params
from pathlib import Path
import matplotlib.pyplot as plt
from skimage.transform import resize
from skimage.io import imread
import matplotlib.patches as patches
%matplotlib qt5

In [3]:
from rtseg.segmentation import LiveNet, get_live_model, live_segment

/home/pk/Documents/rtclient/.venv/lib/python3.11/site-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: '2.0.1' (you have '2.0.0'). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [4]:
params_path = Path("/home/pk/Documents/rtseg/rtseg/resources/reference_params/reference_linux.yaml")
params = load_params(params_path, ref_type='expt')

In [5]:
phase_path = Path('/home/pk/Documents/rtclient/rtclient/resources/test_images/phase_dummy.tiff')
fluor_path = Path('/home/pk/Documents/rtclient/rtclient/resources/test_images/fluor_dummy.tiff')
phase_img = imread(phase_path).astype('float32')
fluor_img = imread(fluor_path)
datapoint = {
    'phase': phase_img,
    'fluor': fluor_img,
    'position': 1,
    'timepoint': 0,
}

In [6]:
live_net = get_live_model(params)

In [7]:
seg_result = live_segment(datapoint, live_net, params)

In [8]:
plt.figure()
plt.imshow(seg_result['seg_mask'])
plt.show()

In [9]:
plt.figure()
plt.imshow(phase_img)
plt.show()

In [10]:
fig, ax = plt.subplots()
ax.imshow(phase_img, cmap='gray')
for row in seg_result['barcode_locations']:
    rect = patches.Rectangle((row[0], row[1]), row[2] - row[0], row[3] - row[1], linewidth=1, edgecolor='r', facecolor='none')
    ax.add_patch(rect)
plt.show()

In [11]:
seg_result

{'phase': array([[ 9191,  9630,  9750, ...,  9215,  9126,  8947],
        [ 9543,  9264,  9978, ...,  8928,  8727,  9221],
        [ 9442,  9854,  9719, ...,  9542,  9192,  9502],
        ...,
        [ 9132, 10096, 10089, ..., 10937, 10458, 10586],
        [ 9916,  9450,  9832, ..., 10307, 10652, 11273],
        [ 9788,  9445, 10127, ..., 10408, 10779, 10595]], dtype=uint16),
 'fluor': array([[251, 201, 266, ..., 241, 200, 229],
        [222, 220, 231, ..., 211, 221, 218],
        [215, 220, 243, ..., 218, 222, 226],
        ...,
        [253, 260, 225, ..., 207, 244, 213],
        [275, 195, 236, ..., 201, 239, 236],
        [273, 254, 220, ..., 195, 215, 218]], dtype=uint16),
 'position': 1,
 'timepoint': 0,
 'barcode_locations': [array([5.03924990e+00, 2.19385025e+02, 9.49055023e+01, 1.00268695e+03,
         6.10479295e-01, 0.00000000e+00], dtype=float32),
  array([1.4597417e+03, 3.3374533e+02, 1.5652759e+03, 1.1148585e+03,
         6.3969195e-01, 0.0000000e+00], dtype=float32),
  

In [12]:
from rtseg.dotdetect import compute_dot_coordinates

In [13]:
params.Hardware.device

'cuda:0'

In [15]:
dots= compute_dot_coordinates(seg_result['fluor'], seg_result['seg_mask'], params)

In [16]:
dots

array([[ 783.14738426, 1487.63868987],
       [1265.01042238, 3027.50923386],
       [ 277.35200941, 2939.82150615],
       ...,
       [1014.54571118,  100.4640608 ],
       [ 168.5046111 ,   19.1091004 ],
       [ 169.431315  ,    9.47310662]])

In [17]:
def plot_dots(fluor_image, dots):
    plt.figure()
    plt.imshow(fluor_image, cmap='gray')
    plt.plot(dots[:, 1], dots[:, 0], 'ro')
    plt.show()

In [18]:
plot_dots(seg_result['fluor'], dots)